# Summary

Cleaning data is a tedious, often manual process. While some steps can be automated, often we will need to find the discrepancies between the expected data (format, content...) and the actual data.  Common issues include:
- Wrong data type
    - Integers loaded as floats (pandas issue).
    - Including float rounding differences.
    - Dates not loaded as dates.    
    
- Data not conforming to data type
    - Isolated values with different data types vs the rest of their column
    - Wrong numeric data (dollar sign included, decimal, separator)
    
- Wrong input values  
    - Wrong number format (Year loaded as 17 instead of 2017)
    - Repeated strings, similar
   


    

    - 
- Missing data
    - Emtpy values
    - Unknown, null and other strings
- Duplicates
- Simplification of data


For example, we might have a column with Euro values saved as a number (e.g. `1000`) but some values stored with the Euro sign (`€1000`). In that case, if we want to keep the column as a number, we need to remove the `€`.

Objective:
- Enough data for analysis
- Reduce the number of years to keep file small
- Eliminate non-core columns with limited data (many nulls)

> What is difficult, it is manual

> Load the parties_data file, only if no local copy exists

> create this file without being "corrected"

# Initial load

In [1]:
file_name = 'TCCSF_parties_data.csv'

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(file_name)

C:\Users\dirkg\AppData\Local\Temp\ipykernel_3804\707990406.py:1: DtypeWarning: Columns (3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


Importing the data without further arguments can lead to some errors or warnings. Here, Pandas is complaining about some specific column types. When loading a dataset, Pandas will try guessing the data type of each column. To avoid the warning, we can force the data type.

> Add more info about the error.

In [4]:
df.columns[[3, 15]]

Index(['case_id_pkey', 'juris'], dtype='object')

Those columns were supposed to be numbers. The error is raised when Pandas encounters just a few values that do not match the type that could suit most of the values. In those cases, we load the data as strings, and later analyse and fix the issues.

In [5]:
df.iloc[:5,3]

0    131045181
1      2008108
2      5469799
3    210012766
4      5590027
Name: case_id_pkey, dtype: object

Short exploration confirms it. Is that always the case? If dtype is object, it is likely there is at least one row that has a value that couldn't be converted to an integer. Let's explore which one, how many...

In [6]:
forced_types = {'case_id_pkey': str, 'juris': str}
df = pd.read_csv(file_name, dtype=forced_types)

If we want, we can also select a specific column (or set of columns) to be the index. In this case, since `unique_id` (column 0) is the primary key of the data, we will load it as index:

In [7]:
df = pd.read_csv(file_name, index_col=0, dtype={'case_id_pkey': str, 'juris': str})

One more thing, you might work with datasets containing characters such as ñ, ü, è, etc. that require a different encoding. If that is the case (not here) you need to select the right encoding when using `read_csv`.

In [8]:
df.columns

Index(['cnn_intrsctn_fkey', 'cnn_sgmt_fkey', 'case_id_pkey', 'tb_latitude',
       'tb_longitude', 'geocode_source', 'geocode_location',
       'collision_datetime', 'collision_date', 'collision_time',
       'accident_year', 'month', 'day_of_week', 'time_cat', 'juris',
       'officer_id', 'reporting_district', 'beat_number', 'primary_rd',
       'secondary_rd', 'distance', 'direction', 'weather_1', 'weather_2',
       'collision_severity', 'type_of_collision', 'mviw', 'ped_action',
       'road_surface', 'road_cond_1', 'road_cond_2', 'lighting',
       'control_device', 'intersection', 'vz_pcf_code', 'vz_pcf_group',
       'vz_pcf_description', 'vz_pcf_link', 'number_killed', 'number_injured',
       'street_view', 'dph_col_grp', 'dph_col_grp_description',
       'party_number_ckey', 'party_type', 'at_fault', 'party_sex', 'party_age',
       'party_sobriety', 'party_drug_physical', 'dir_of_travel',
       'party_safety_equip_1', 'party_safety_equip_2', 'finan_respons',
       'sp_inf

# Drop unnecessary data

To avoid working with unnecessarily big datasets, we should first drop the data we do not need. 

In [9]:
drop_columns = ['geocode_source', # Source of Lat/Lon; not needed for the project
                'geocode_location',  # Source of Lat/Lon; not needed for the project
                'intersection', # Not needed. we can use distance and type_of_colision
                'juris', # Jurisdiction not useful for project analysis
                'reporting_district', # Multiple values used one; doesn't add to analysis
                'beat_number', # Multiple values used one; doesn't add to analysis
                'SF Find Neighborhoods', # Not keeping neighborhood data (too granular + definitions not 100% clear)
                'Neighborhoods', # Not keeping neighborhood data (too granular + definitions not 100% clear)
                'Analysis Neighborhoods', # Not keeping neighborhood data (too granular + definitions not 100% clear)
                'analysis_neighborhood', # Not keeping neighborhood data (too granular + definitions not 100% clear)
                'data_as_of', # For the analysis we don't need metadata on the entries
                'data_updated_at', # For the analysis we don't need metadata on the entries
                'data_loaded_at', # For the analysis we don't need metadata on the entries
                'point', # Alternative to lat/Lon
                'collision_date', # Redundant when having collision_datetime
                'collision_time', # Redundant when having collision_datetime
                'accident_year', # Redundant when having collision_datetime
                'month', # Redundant when having collision_datetime
                'day_of_week', # Redundant when having collision_datetime
                'vz_pcf_code', # vz_pcf_description is enough
                'vz_pcf_group', # vz_pcf_description is enough
                'vz_pcf_link', # vz_pcf_description is enough
                'number_killed', # redundant since we have party_number_killed
                'number_injured', # redundant since we have party_number_injured
                'street_view', # Can be built with lat/lon
                'dph_col_grp', # dph_col_grp_description is enough
                'cnn_intrsctn_fkey', # Cannot really relate it to the primary road, and no table with this as pkey
                'cnn_sgmt_fkey',
               'supervisor_district', # old district info
               'police_district',  # old district info
                'time_cat' # 4h-group of time of the day; redundant
               ]

In [10]:
df.drop(columns=drop_columns, inplace=True)
df.shape

(128027, 56)

We could also have simply not loaded the data when reading from the CSV using a parameter for `read_csv`. Make sure to add those columns you want to use as index to the list.

In [11]:
useful_columns = ['unique_id', 'case_id_pkey', 'tb_latitude', 'tb_longitude', 'collision_datetime',
       'officer_id', 'primary_rd', 'secondary_rd', 'distance', 'direction',
       'weather_1', 'weather_2', 'collision_severity', 'type_of_collision',
       'mviw', 'ped_action', 'road_surface', 'road_cond_1', 'road_cond_2',
       'lighting', 'control_device', 'vz_pcf_description', 'dph_col_grp_description', 'party_number_ckey',
       'party_type', 'at_fault', 'party_sex', 'party_age', 'party_sobriety',
       'party_drug_physical', 'dir_of_travel', 'party_safety_equip_1',
       'party_safety_equip_2', 'finan_respons', 'sp_info_1', 'sp_info_2',
       'sp_info_3', 'oaf_viol_cat', 'oaf_viol_section', 'oaf_violation_code',
       'oaf_violation_suffix', 'oaf_1', 'oaf_2', 'party_number_killed',
       'party_number_injured', 'move_pre_acc', 'vehicle_year', 'vehicle_make',
       'stwd_vehicle_type', 'race', 'inattention', 'special_info_f',
       'special_info_g', 'street_of_travel', 'vehicle_autonomous',
       'Current Police Districts', 'Current Supervisor Districts']

In [12]:
df = pd.read_csv(file_name, index_col='unique_id', dtype={'case_id_pkey': str}, usecols=useful_columns)
df.shape

(128027, 56)

In any case, it is useful to know that columns can be dropped from the dataset. We might want to use their data as part of the cleaning process, but remove them at the end.

# Fix format

We will first fix the data format of the remaining columns. Pandas will default to type `object`, that is a set of characters. We could have forced some data types at the beginning when loading the data, but the advantage of doing it later is that we can inspect the data. In some cases we won't be able to set the desired type because of wrong data. For example, we might have a column with Euro values saved as a number (e.g. `1000`) but some values stored with the Euro sign (`€1000`). In that case, if we want to keep the column as a number, we need to remove the `€`. Let's look at all the types:

## Columns that should be integers

> Integrated the following code in each section

int_columns = ['Current Supervisor Districts', 
               'Current Police Districts', 
               #'distance', 
               #'number_killed', 
               'vehicle_year',
               #'party_age']
#   df[column] = df[column].astype(pd.Int64Dtype())

# Issues with columns

## Column `case_id_pkey`

This is a case of having isolated values with different data types vs the rest of their column. First, let's identify those cases that do not conform to our expectations. Pandas wanted to convert to numbers, yet it found a few values that couldn't (hence the warning with `read_csv`).

In [13]:
mask_non_numeric = pd.to_numeric(df['case_id_pkey'], errors='coerce').isna()
mask_non_numeric.sum()

4

In [14]:
df.shape[0]

128027

That is only 4 cases out of 128.027 rows.

In [15]:
values_non_numeric = list(df['case_id_pkey'][mask_non_numeric].unique())
values_non_numeric

['210072867`', '9335-2020-00557']

As we see, we have 4 rows, but only 2 different issues. In the first case, a non-digit character is added at the end, likely a mistake. In the second, dashes are added as probably that is how the case ID is sometimes formatted. An obvious option here is to remove the non-digit characters, but before that, we should verify if any rows exist with the resulting values, and if they do, if they "look" like they are the same case. 

In [16]:
modified_values = df['case_id_pkey'][mask_non_numeric].str.replace(r'\D+', '', regex=True)
values_numeric = list(modified_values.unique())
values_numeric

['210072867', '9335202000557']

In [17]:
mask_numeric = df['case_id_pkey'].isin(values_numeric)
mask_numeric.sum()

2

In [18]:
list(df['case_id_pkey'][mask_numeric].unique())

['210072867']

There are two rows that already contain `case_id_pkey` equal to `210072867`. Let's change the values to remove the non-digit values, and then verify how the rows with that case number look like.

In [19]:
indexes = modified_values.index
columns = ['case_id_pkey']
df.loc[indexes, columns] = modified_values

In [20]:
df[df['case_id_pkey'] == '210072867'].duplicated()

unique_id
627345    False
627346    False
632567    False
632566    False
dtype: bool

After removing from the `case_id_pkey` the non-digit character, the rows for case `210072867` are duplicated. No further action is required now, since we will remove duplicated rows at a later stage.

## Columns `tb_latitude` and `tb_longitude`

An issue with `float` numbers is that they might have too many decimal points for our needs, and two numbers that should be the same are stored differently. For example, in this data set we have latitude and longitude. We don't need more than 6 decimals to have a decent precision. Let's look at an example:

In [21]:
study_indexes = [627345, 632567]
df.loc[study_indexes, 'tb_latitude']

unique_id
627345    37.769483
632567    37.769483
Name: tb_latitude, dtype: float64

In [22]:
df.loc[study_indexes, 'tb_latitude'].value_counts()

tb_latitude
37.769483    1
37.769483    1
Name: count, dtype: int64

They look the same, right? They are to you, but internally there are small differences that Python is not showing when rounding the number for display...

In [23]:
weird_values = df.loc[study_indexes, 'tb_latitude'].unique()
weird_values[0] - weird_values[1]

3.416289473534562e-11

The difference is smaller than one millionth of a degree. It is, in fact, a data issue. To avoid this, we will round the two columns and check that the issue has been resolved:

In [24]:
for column in ['tb_latitude', 'tb_longitude']:
    df[column] = df[column].round(6)

In [25]:
df.loc[study_indexes, 'tb_latitude'].value_counts()

tb_latitude
37.769483    2
Name: count, dtype: int64

We have already fixed part of the issue with this column by rounding the `float` values. We should verify that the values are correct, and no point is far outside the city of San Francisco. An option would be to plot the points, but to avoid installing additional packages, we will simply check the minimum and maximum values.

The approximate boundaries of the city are:

In [26]:
max_lat = 37.8324
min_lat = 37.7081
max_lon = -122.3570
min_lon = -123.1738

In [27]:
df['tb_latitude'].describe()['max'] < max_lat

True

In [28]:
df['tb_latitude'].describe()['min'] > min_lat

False

In [29]:
df['tb_latitude'].describe()['min'] 

37.707459

In [30]:
df[df['tb_latitude']== 37.707459]['tb_longitude'].drop_duplicates().values

array([-122.4629])

There is a small discrepancy between our boundary and the smallest number. If we check the coordinates we'll find that the address corresponds to Daly City, just south of San Francisco.

In [31]:
df['tb_longitude'].describe()['max'] < max_lon

True

In [32]:
df['tb_longitude'].describe()['min'] > min_lon

True

## Column `collision_datetime`

In [33]:
df['collision_datetime'].head(5)

unique_id
36115     12/12/2013 02:05:00 PM
2178      04/14/2005 04:20:00 PM
28980     01/22/2012 07:46:00 PM
627682    01/06/2021 05:45:00 PM
29244     02/22/2012 10:02:00 PM
Name: collision_datetime, dtype: object

As we can observe, the string stores de day, month, year and time of the incident. In case we need to manipulate this as a date, it is useful to transform it to a `datetime64[ns]` format. This will be useful only while working with Pandas, after we export the clean CSV file, the date will be stored as a string.

In [34]:
df['collision_datetime'] = pd.to_datetime(df['collision_datetime'], format='%m/%d/%Y %I:%M:%S %p')
df['collision_datetime'].head(5)

unique_id
36115    2013-12-12 14:05:00
2178     2005-04-14 16:20:00
28980    2012-01-22 19:46:00
627682   2021-01-06 17:45:00
29244    2012-02-22 22:02:00
Name: collision_datetime, dtype: datetime64[ns]

In [35]:
df['collision_datetime'].describe()[['min', 'max']]

min    2005-01-01 01:15:00
max    2024-10-31 22:00:00
Name: collision_datetime, dtype: object

Data starts on year 2005... and should end July 2024, when the data was extracted. Here, we could limit our data to the date when we extracted the data. However, since we want to reduce the size of the output file to less than 25MB (so we can upload to GitHub), we will need to further reduce the size.

In [36]:
min_year = 2016
max_year = 2023
df = df[(df['collision_datetime'].dt.year >= min_year) & (df['collision_datetime'].dt.year <= max_year)]
df.shape

(51531, 56)

We can easily revisit the upper and lower limit boundary of our dataset if at the end of the process the file is still too large.

## Column `officer_id`

In [37]:
df['officer_id'].value_counts()

officer_id
2375           283
1994           270
4314           242
2132           201
4286           195
              ... 
1785             1
#2732            1
824              1
GODOY#02202      1
4225             1
Name: count, Length: 2086, dtype: int64

We see over 2000 different officer ID values. Just looking at the values printed on screen we could guess some issues. For example, there is an officer with id `#2732`. The hash is often used as a replacement for the word "number", so... do we have any reports from officer `2732` (without hash)?

In [38]:
(df['officer_id'] == '2732').sum()

40

Yes. We do. Let's see how many officer ids contain the non-numeric values.

In [39]:
mask_contains_other = ~ (df['officer_id'].str.isdigit().fillna(True))

In [40]:
df['officer_id'][mask_contains_other].value_counts()

officer_id
#1118         97
#2008         78
#2450         41
#2035         24
#1500         23
              ..
#1533          1
#2359          1
171 & 1387     1
#2732          1
#364           1
Name: count, Length: 417, dtype: int64

That is 417 out of 2086 different values. This inspection shows two things:
1. Several cases include the `#`. We should remove it to have the same format.
2. There are cases that the user entered two numbers: `171 & 1387`. For now, let's treat those as an independent id.

In [41]:
df['officer_id'] = df['officer_id'].str.replace('#','')

In [42]:
mask_contains_other = ~ (df['officer_id'].str.isdigit().fillna(True))

In [43]:
df['officer_id'][mask_contains_other].value_counts()

officer_id
A22028        21
Li 1074       20
A20952        20
A09968        19
A23073        17
              ..
HAWES 4281     1
ANTON 2179     1
A20831         1
A07632         1
A22397         1
Name: count, Length: 274, dtype: int64

New cases appear: officers write their name before the number, or an "A". Can we assume that the letter A is redundant here? Should those officers without an A have written the A? When doing data cleaning, often we need knowledge not available in our data.

We will *assume* that any non-number before the number is not part of the officer ID. Not all of them though, as we don't want to eliminate the spaces and "&" between the case `171 & 1387` and turn it into a new number (`1711387`).

In [44]:
modified_values = df['officer_id'][mask_contains_other].str.replace(r'^\D*(?=\d)', '', regex=True)
indexes = modified_values.index
columns = ['officer_id']
df.loc[indexes, columns] = modified_values

In [45]:
mask_contains_other = ~ (df['officer_id'].str.isdigit().fillna(True))

In [46]:
df['officer_id'][mask_contains_other].value_counts()

officer_id
Li                10
Chicas, M.         9
ALVENDIA           7
FYLES              7
P. Rechsteiner     7
                  ..
HERERRA            2
VARGAS             2
20:22              2
3E15D              2
171 & 1387         1
Name: count, Length: 88, dtype: int64

In [47]:
df['officer_id'].value_counts().shape[0]

1846

Since there were some cases with name and number (like `Li 1074`) we could try to further link those values; we could replace those ids only with name, with the number (for example `Li` becomes `1074`). However, not having a full database we could be confusing two officers with the same name. Hence, we stop here. We have reduced the number of different values to 1846, and only 88 of those are not formatted as a just a number.

We check if there are any strings that indicate no value:

In [48]:
df['officer_id'].str.title().isin(['Unknown', 'Not Stated']).sum()

0

## Column `primary_rd`

In [49]:
df['primary_rd'].value_counts()

primary_rd
MISSION ST               2296
MARKET ST                1504
VAN NESS AVE             1127
03RD ST                  1059
GEARY BLVD                866
                         ... 
WETMORE ST                  1
HIGUERA AVE                 1
LECH WALESA ST              1
SAN JOSE AVE OFF RAMP       1
ROSENKRANZ ST               1
Name: count, Length: 947, dtype: int64

In [50]:
df['primary_rd'].value_counts().tail(10)

primary_rd
GELLERT DR               1
BELGRAVE AVE             1
CERRITOS AVE             1
RIVAS AVE                1
LASKIE ST                1
WETMORE ST               1
HIGUERA AVE              1
LECH WALESA ST           1
SAN JOSE AVE OFF RAMP    1
ROSENKRANZ ST            1
Name: count, dtype: int64

Looking at the `value_counts()`, we observe 947 different street names. Issues aren't obvious. Even looking the last 10 cases we don't see an immediate big issue. Two things stand out:
1. `GELLERT DR` has no `ST` or `AVE` in it, unlike all other options.
2. `SAN JOSE AVE OFF RAMP` has the "off ramp" part that comes after avenue. Might there be a San Jose Avenue without the offramp?

Let's start investigating there.

In [51]:
mask_gellert = df['primary_rd'].str.contains('GELLERT')
df['primary_rd'][mask_gellert].value_counts()

primary_rd
GELLERT DR    1
Name: count, dtype: int64

In [52]:
mask_san_jose = df['primary_rd'].str.contains('SAN JOSE')
df['primary_rd'][mask_san_jose].value_counts()

primary_rd
SAN JOSE AVE             444
SAN JOSE AV OFF RAMP       3
SAN JOSE AVE OFF RAMP      1
Name: count, dtype: int64

In [53]:
mask_off_ramp = df['primary_rd'].str.contains('OFF RAMP')
df['primary_rd'][mask_off_ramp].value_counts()

primary_rd
HWY 101 S OFF RAMP              12
BAY SHORE BLVD OFF RAMP          7
JUNIPERO SERRA BLVD OFF RAMP     6
I-280 N OFF RAMP                 4
HWY 101 N OFF RAMP               4
I-280 S OFF RAMP                 3
ALEMANY BLVD OFF RAMP            3
SAN JOSE AV OFF RAMP             3
INDUSTRIAL ST OFF RAMP           1
SAN JOSE AVE OFF RAMP            1
Name: count, dtype: int64

There seems to be no issue with `GELLERT`, but we observe that several streets names also refer to the off ramp. The choice of adding `OFF RAMP` is repeated a few times, and it adds additional information. Thus, we will leave it as it is.

In [54]:
mask_av_only = df['primary_rd'].str.contains(' AV') & (~df['primary_rd'].str.contains(' AVE'))
df['primary_rd'][mask_av_only].value_counts()

primary_rd
SAN JOSE AV OFF RAMP    3
Name: count, dtype: int64

The typo in `SAN JOSE AV OFF RAMP` (missing an "E") only happens for one road.

In [55]:
df['primary_rd'] = df['primary_rd'].str.replace('SAN JOSE AV OFF RAMP', 'SAN JOSE AVE OFF RAMP')
df['primary_rd'][mask_san_jose].value_counts()

primary_rd
SAN JOSE AVE             444
SAN JOSE AVE OFF RAMP      4
Name: count, dtype: int64

A more scalable way to check this is finding if there are any combinations of street names that differ slightly. We will exclude those streets that are simply a number street/avenue:

In [56]:
df_study = df['primary_rd'].copy().drop_duplicates()
for elem in ['ST ST', 'ND ST', 'RD ST', 'TH ST', 'ST AVE', 'ND AVE', 'RD AVE', 'TH AVE']:
    pattern = rf'\b\d{{2}}{elem}\b'
    index = df_study[df_study.str.contains(pattern, na=False)].index
    df_study.drop(index, inplace=True)    

We can control the similarity using the `threshold`. The closer it is to 1, the more similar the stirngs must be. Given that some names are very similar despite being different streets (like one street containing the word "east" and another "west"). There are other mechanisms to find similarities. In a case like this, we would very likely need some manual checking, or at least using additional map libraries to check the coordinates. The code below is shared as an example, but the values are not further changed.

In [57]:
import difflib
for i in range(len(df_study)):
    for j in range(i+1, len(df_study)):
        ratio = difflib.SequenceMatcher(None, df_study.iloc[i], df_study.iloc[j]).ratio()
        if ratio > 0.9 and ratio < 1:
            print([df_study.iloc[i], df_study.iloc[j]])

['MISSION ST', 'MISSION ST.']
['NORTH POINT ST', 'NORTHPOINT ST']
['ALEMANY BLVD WEST', 'ALEMANY BLVD EAST']
['ALEMANY BLVD', 'AELMANY BLVD']
['TERRY A FRANCOIS BLVD', 'TERRY FRANCOIS BLVD']
['HWY 101 S OFF RAMP', 'HWY 101 S ON RAMP']
['HWY 101 S OFF RAMP', 'HWY 101 N OFF RAMP']
['09TH TI ST', '9TH TI ST']
['HWY 101 N ON RAMP', 'HWY 101 S ON RAMP']
['HWY 101 N ON RAMP', 'HWY 101 N OFF RAMP']
['MISSION BAY BLVD NORTH', 'MISSION BAY BLVD SOUTH']
['JUNIPERO SERRA BLVD OFF RAMP', 'JUNIPERO SERRA BLVD ON RAMP']
['JUNIPERO SERRA BLVD OFF RAMP', 'JUNIPERO SERRA  BLVD ON RAMP']
['LAKE ST', 'BLAKE ST']
['BAY SHORE BLVD EXT', 'BAY SHORE BLVD EAST']
['EL CAMINO DEL MAR', 'CAMINO DEL MAR']
['MISSION BAY SOUTH BLVD', 'MISSION BAY NORTH BLVD']
['TENNESSEE ST', 'GENNESSEE ST']
['MARIN ST', 'MAIN ST']
['I-280 S OFF RAMP', 'I-280 S ON RAMP']
['I-280 S OFF RAMP', 'I-280 N OFF RAMP']
['I-280 S ON RAMP', 'I-280 N ON RAMP']
['CHESTER AVE', 'HESTER AVE']
['EDINBURGH ST', 'EDINBURG ST']
['JUNIPERO SERRA BLVD

We check if there are any strings that indicate no value:

In [58]:
(df['primary_rd'] == 'UNKNOWN').sum()

5

In [59]:
(df['primary_rd'] == 'NOT STATED').sum()

0

In [60]:
df.loc[df['primary_rd'] == 'UNKNOWN', 'primary_rd'] = 'Not Stated'

## Column `secondary_rd`

In [61]:
df['secondary_rd'].value_counts()

secondary_rd
MISSION ST               818
FOLSOM ST                564
HARRISON ST              532
VAN NESS AVE             523
16TH ST                  509
                        ... 
EAGLE ST                   1
WOODACRE DR                1
HILL POINT AVE             1
LIGHT POLE USED AS RP      1
WHITING WAY                1
Name: count, Length: 1425, dtype: int64

We could do the same approach as with the primary road. In this case, the issues are different, so we will only fix the nulls:

In [62]:
df.loc[df['secondary_rd'].str.upper() == 'UNKNOWN', 'secondary_rd'] = 'Not Stated'
df.loc[df['secondary_rd'].str.upper() == 'NOT STATED', 'secondary_rd'] = 'Not Stated'
df.loc[df['secondary_rd'].isna(), 'secondary_rd'] = 'Not Stated'

## Column `distance`

In [63]:
df['distance'].drop_duplicates().sort_values().head(10)

unique_id
626976    0.0
48588     1.0
245780    2.0
88470     3.0
47420     4.0
48178     5.0
87181     6.0
626600    7.0
634125    8.0
88930     9.0
Name: distance, dtype: float64

We will convert the column integers. Pandas chose a `float64` type not because it detected decimals, but because there were null values (see the *not a number* above, or `nan`) as `int64` does not accept nulls. If we integers, we can use `pd.Int64Dtype()`. Having an integer type is useful in case we need to do manipulation during the cleaning process, and to avoid unnecessary bigger files by storing the decimal (always `.0`). 

In [64]:
df['distance'] = df['distance'].astype(pd.Int64Dtype())

In [65]:
df['distance'].drop_duplicates().sort_values().head(10)

unique_id
626976    0
48588     1
245780    2
88470     3
47420     4
48178     5
87181     6
626600    7
634125    8
88930     9
Name: distance, dtype: Int64

In [66]:
df['distance'].describe()

count       51500.0
mean      49.395631
std      155.505221
min             0.0
25%             0.0
50%             0.0
75%            50.0
max         13925.0
Name: distance, dtype: Float64

The maximum distance is 13925 ft. from the intersection. That's roughly 4.2km. That's too far to describe a point from an intersection. Having no guidance on a maximum threshold, we will leave the values as they are.

## Column `direction`

In [67]:
df['direction'].value_counts()

direction
Not Stated    30964
South          5388
East           5173
North          5074
West           4930
North Nor         2
Name: count, dtype: int64

This column is a simple case. More than 50% of the rows do not have a direction from the intersection. We could potentially obtain it from the coordinates and the roads that form the intersection, but that would require bringing further data (the coordinates of the intersection) so we will not do it as part of this process. What we can do, is fix those two cases with value `North Nor`.

In [68]:
df['direction'] = df['direction'].str.replace('North Nor', 'North')
df['direction'].value_counts()

direction
Not Stated    30964
South          5388
East           5173
North          5076
West           4930
Name: count, dtype: int64

Also, we should have no NULLs if we have the category `Not Stated`:

In [69]:
df['direction'].isna().sum()

0

## Columns `weather_1` and `weather_2`

In [70]:
df['weather_1'].value_counts()

weather_1
Clear                       43366
Cloudy                       4027
Raining                      2871
Not Stated                    791
Other                         240
Fog                           173
Wind                           48
Fog / Visibility: 10 ft         4
Other: NOT AT SCENE             2
Other: MISTING                  2
Fog / Visibility: 800 ft        2
Other: Unknown                  2
Other: NOT ON SCENE             2
Fog / Visibility                1
Name: count, dtype: int64

In [71]:
df['weather_2'].value_counts()

weather_2
Not Stated                     50159
Raining                          410
Wind                             135
Cloudy                           109
Fog / Visibility                  16
Other                             10
Fog                                6
Other: Drizzling                   4
Other: MIST                        4
Other: Night                       3
Other: FALLING ASH                 3
Other: SUNRISE                     2
Other: Mist                        2
Fog / Visibility: 500 ft           2
Other: SUNSET                      2
Other: smoke cloud                 2
Other: Sunny                       2
Fog / Visibility: 15 ft            2
Snowing                            2
Other: NOT ON SCENE                2
Other: Sunset                      2
Fog / Visibility: 30 ft            2
Other: SMOKEY SKIES                2
Other: HEAVY MIST                  2
Fog / Visibility: 100 ft           2
Other: SUN IN WB VIEW              2
Other: Light drizzle (rain) 

In [72]:
df['weather_1'].isna().sum()

0

In [73]:
df['weather_2'].isna().sum()

631

For some reason, a large variety of infrequent responses appear starting with `Other: ` and `Fog /`. We will consolidate those values in a single category. If the values were more frequent, it would be worth leaving them as they are, given that some contain addition small pieces of information.

Also, it makes no sense to have a value being `Not Stated` and then having empty cells (NULL). We will tag all nulls as "Not Stated".

In [74]:
for column in ['weather_1', 'weather_2']:
    df.loc[df[column].isna(), column] = 'Not Stated'
    df.loc[df[column].str.contains('Fog'), column] = 'Fog'
    df.loc[df[column].str.contains('Other'), column] = 'Other'

In [75]:
df['weather_1'].value_counts()

weather_1
Clear         43366
Cloudy         4027
Raining        2871
Not Stated      791
Other           248
Fog             180
Wind             48
Name: count, dtype: int64

In [76]:
df['weather_2'].value_counts()

weather_2
Not Stated    50790
Raining         410
Wind            135
Cloudy          109
Other            53
Fog              32
Snowing           2
Name: count, dtype: int64

## Column `collision_severity`

In [77]:
df['collision_severity'].value_counts()

collision_severity
Injury (Complaint of Pain)    33298
Injury (Other Visible)        13687
Injury (Severe)                4056
Fatal                           485
Medical                           5
Name: count, dtype: int64

If we look at the SWITRS classification of the fields, we expect three levels of injury and fatal as categories. `Medical` does not fit any category, so we cannot reclassify it. We will leave it as it is.

## Column `type_of_collision`

In [78]:
df['type_of_collision'].value_counts()

type_of_collision
Broadside             16124
Vehicle/Pedestrian     9738
Rear End               8622
Sideswipe              7843
Head-On                3346
Other                  2266
Not Stated             1636
Hit Object             1226
Overturned              730
Name: count, dtype: int64

In [79]:
df['type_of_collision'].isna().sum()

0

No strange values in the `type_of_collision` column, and there are no nulls, all values not entered are counted as `Not Stated`.

## Column `mviw`

In [80]:
df['mviw'].value_counts()

mviw
Other Motor Vehicle               24673
Pedestrian                        11329
Bicycle                            6430
Parked Motor Vehicle               3154
Not Stated                         1617
Fixed Object                       1371
Other Object                       1087
Motor Vehicle on Other Roadway      986
Non-Collision                       750
Train                               111
Animal                               23
Name: count, dtype: int64

In [81]:
df['mviw'].isna().sum()

0

No strange values in the `mviw` column, and there are no nulls. Nothing to do here.

## Column `ped_action`

In [82]:
df['ped_action'].value_counts()

ped_action
No Pedestrian Involved                       38884
Crossing in Crosswalk at Intersection         6873
Crossing Not in Crosswalk                     2170
In Road, Including Shoulder                   1816
Not Stated                                     799
Not in Road                                    792
Crossing in Crosswalk Not at Intersection      161
Not In Road                                     28
Approaching/Leaving School Bus                   8
Name: count, dtype: int64

In [83]:
df['ped_action'].isna().sum()

0

Small issue here with two values being `Not in Road`, but one with capital "I". We will replace that one as it is used less frequently.

In [84]:
df['ped_action'] = df['ped_action'].str.replace('Not In Road', 'Not in Road')
df['ped_action'].value_counts()

ped_action
No Pedestrian Involved                       38884
Crossing in Crosswalk at Intersection         6873
Crossing Not in Crosswalk                     2170
In Road, Including Shoulder                   1816
Not in Road                                    820
Not Stated                                     799
Crossing in Crosswalk Not at Intersection      161
Approaching/Leaving School Bus                   8
Name: count, dtype: int64

A different, more scalable approach to this issue, could be turning all values to lower case. If we know that upper case letters are not necessary to differentiate values (as it would in case we had acronyms), we can also use the `.title()`, `.upper()` or `.lower()` methods to format all the strings. While the number of different values won't change now that we have fixed the issue, we display here how it would have looked like:

In [85]:
df['ped_action'].str.upper().value_counts()

ped_action
NO PEDESTRIAN INVOLVED                       38884
CROSSING IN CROSSWALK AT INTERSECTION         6873
CROSSING NOT IN CROSSWALK                     2170
IN ROAD, INCLUDING SHOULDER                   1816
NOT IN ROAD                                    820
NOT STATED                                     799
CROSSING IN CROSSWALK NOT AT INTERSECTION      161
APPROACHING/LEAVING SCHOOL BUS                   8
Name: count, dtype: int64

In [86]:
df['ped_action'].str.title().value_counts()

ped_action
No Pedestrian Involved                       38884
Crossing In Crosswalk At Intersection         6873
Crossing Not In Crosswalk                     2170
In Road, Including Shoulder                   1816
Not In Road                                    820
Not Stated                                     799
Crossing In Crosswalk Not At Intersection      161
Approaching/Leaving School Bus                   8
Name: count, dtype: int64

## Column `road_surface`

In [87]:
df['road_surface'].value_counts()

road_surface
Dry             45202
Wet              4952
Not Stated       1292
Slippery           55
Snowy or Icy       30
Name: count, dtype: int64

In [88]:
df['road_surface'].isna().sum()

0

No issues with `road_surface`.

## Columns `road_cond_1` and `road_cond_2`

In [89]:
df['road_cond_1'].value_counts()

road_cond_1
No Unusual Condition           48185
Not Stated                      1855
Construction or Repair Zone      541
Other                            508
Obstruction on Roadway           143
Holes, Deep Rut                   99
Holes, Deep Ruts                  77
Loose Material on Roadway         67
Reduced Roadway Width             47
Flooded                            9
Name: count, dtype: int64

In [90]:
df['road_cond_2'].value_counts()

road_cond_2
Not Stated                     51408
No Unusual Condition              56
Construction or Repair Zone       27
Reduced Roadway Width             25
Other                              7
Obstruction on Roadway             7
Loose Material on Roadway          1
Name: count, dtype: int64

In [91]:
df['road_cond_1'].isna().sum()

0

In [92]:
df['road_cond_2'].isna().sum()

0

There are two spellings of `Holes, Deep Ruts`, one with `s` and one without. We cannot use `.str.replace('Holes, Deep Rut', 'Holes, Deep Ruts')` because then the value with `s` will have two letter `s` at the end.

In [93]:
df.loc[df['road_cond_1'] == 'Holes, Deep Rut', 'road_cond_1'] = 'Holes, Deep Ruts'

In [94]:
df['road_cond_1'].value_counts()

road_cond_1
No Unusual Condition           48185
Not Stated                      1855
Construction or Repair Zone      541
Other                            508
Holes, Deep Ruts                 176
Obstruction on Roadway           143
Loose Material on Roadway         67
Reduced Roadway Width             47
Flooded                            9
Name: count, dtype: int64

## Column `lighting`

In [95]:
df['lighting'].value_counts()

lighting
Daylight                                32713
Dark - Street Lights                    15815
Dusk - Dawn                              1866
Not Stated                                647
Dark - No Street Lights                   388
Dark - Street Lights Not Functioning      102
Name: count, dtype: int64

In [96]:
df['lighting'].isna().sum()

0

No issues with column `lighting`.

## Column `control_device`

In [97]:
df['control_device'].value_counts()

control_device
Functioning        32701
Not Stated          1298
Not Functioning      196
Obscured              44
Name: count, dtype: int64

In [98]:
df['control_device'].isna().sum()

17292

We need to fix the issue of having `Not Stated` values in the column `control_device` while having nulls:

In [99]:
df.loc[df['control_device'].isna(), 'control_device'] = 'Not Stated'

In [100]:
df['control_device'].value_counts().loc['Not Stated']

18590

In [101]:
df['control_device'].isna().sum()

0

## Column `vz_pcf_description`

In [102]:
df['vz_pcf_description'].value_counts()

vz_pcf_description
Unsafe speed for prevailing conditions                               8902
Red signal - driver or bicyclist responsibilities                    5066
Driver or bicyclist to yield right-of-way at crosswalks              5063
Unsafe turn or lane change prohibited                                4378
Unknown                                                              3160
                                                                     ... 
Dumping material on highway or right-of-way prohibited                  2
Refusal to present license to officer                                   1
Headsets, ear plugs, or earphones in or over both ears prohibited       1
Motorized scooter operation requirements                                1
Blind pedestrians right-of-way                                          1
Name: count, Length: 181, dtype: int64

In [103]:
df['vz_pcf_description'].isna().sum()

0

We observe the value `Unknown`, when we used to have `Not Stated` when the officer didn't select any category.

In [104]:
df['vz_pcf_description'].value_counts().loc['Not Stated']

47

There are a few `Not Stated` too. We will convert all to that value.

In [105]:
df.loc[df['vz_pcf_description'] == 'Unknown', 'vz_pcf_description'] = 'Not Stated'

In [106]:
df['vz_pcf_description'].value_counts().loc['Not Stated']

3207

Also, this column has more values than are shown by default, and they are descriptions. We will use again the similarity search to consolidate some categories.

In [107]:
df_study = df['vz_pcf_description'].drop_duplicates()
for i in range(len(df_study)):
    for j in range(i+1, len(df_study)):
        ratio = difflib.SequenceMatcher(None, df_study.iloc[i], df_study.iloc[j]).ratio()
        if ratio > 0.85 and ratio < 1:
            print([df_study.iloc[i], df_study.iloc[j]])

['Red signal - driver or bicyclist responsibilities', 'Green signal - driver or bicyclist responsibilities']
['Red signal - driver or bicyclist responsibilities', 'Red signal - driver or bicyclist responsibilities with right turn']
['Red signal - pedestrian responsibilities', 'Green signal - pedestrian responsibilities']
['Failure to yield to emergency vehicle', 'Failure to yield to emergency vehicle - pedestrian']
['Actions required at flashing red signal', 'Actions required at flashing yellow signal']
['Operating a non-motorized user-propelled vehicle in a reckless manner', 'Local ordinance, operating a non-motorized user-propelled vehicle in a reckless manner']
['Local ordinance, riding a non-motorized user-propelled vehicle in the roadway', 'Riding a non-motorized user-propelled vehicle in the roadway']


We observe two type of differences in the values:
1. Those that the values are similar, but there is a difference in the situation (like the color of a light).
2. Those that have the same description, but are violations of different laws (local vs state, ...)

In [108]:
for e in df_study[df_study.str.contains("ordinance")].sort_values():
    print(e)

Local ordinance, bicycles prohibited from sidewalk
Local ordinance, driving in transit-only area
Local ordinance, non-motorized user-propelled vehicle - unspecified violation
Local ordinance, operating a non-motorized user-propelled vehicle in a reckless manner
Local ordinance, pedestrian crossing at an inappropriate location
Local ordinance, riding a non-motorized user-propelled vehicle in the roadway
Local ordinance, riding non-motorized user-propelled vehicles in a business district within the City
Local ordinance, sleeping on sidewalk prohibited


Give than this fact could be relevant for the analysis, we keep the data as it is. If the user analysing the data wants to consider the primary collision factor the same regardless of the ordinance, they can still group values. Remember we removed three columns that expanded on this descriptions. We could recover them in case we want to dive deeper.

## Column `dph_col_grp_description`

In [109]:
df['dph_col_grp_description'].value_counts()

dph_col_grp_description
Vehicle(s) Only Involved                    27024
Vehicle-Pedestrian                          10538
Vehicle-Bicycle                              5748
AA                                           4593
BB                                           1567
CC                                            786
Bicycle Only                                  482
Bicycle-Pedestrian                            360
Bicycle-Parked Car                            215
FF                                             73
DD                                             47
EE                                             24
Vehicle-Bicycle-Pedestrian                     24
Pedestrian Only or Pedestrian-Parked Car       24
Bicycle-Unknown/Not Stated                      6
BB CC                                           6
Unknown/Not Stated                              5
GG                                              3
II                                              2
Name: count, dtype: int64

In [110]:
df['dph_col_grp_description'].isna().sum()

4

It's not clear what the values `AA`, `BB`, etc. are. Most likely, internal codes that do not appear in the two sources we have for descriptions. We will only fix the `Not Stated` cases.

In [111]:
df.loc[df['dph_col_grp_description'].isna(), 'dph_col_grp_description'] = 'Not Stated'

In [112]:
df.loc[df['dph_col_grp_description'] == 'Unknown/Not Stated', 'dph_col_grp_description'] = 'Not Stated'

In [113]:
df['dph_col_grp_description'].value_counts().loc['Not Stated']

9

In [114]:
df['dph_col_grp_description'].isna().sum()

0

## Column `party_number_ckey`

In [115]:
df['party_number_ckey'].value_counts().sort_index()

party_number_ckey
1     24563
2     22637
3      3026
4       804
5       292
6       110
7        47
8        21
9        14
10       10
11        3
12        1
13        1
14        1
15        1
Name: count, dtype: int64

In [116]:
df['party_number_ckey'].isna().sum()

0

No issues, but an observation: there is an accident with 15 parties involved.

## Column `party_number_ckey`

In [117]:
df['party_type'].value_counts()

party_type
Driver            36909
Pedestrian         6248
Bicyclist          4091
Parked Vehicle     3147
Other              1136
Name: count, dtype: int64

In [118]:
df['party_type'].isna().sum()

0

No data issues here.

## Column `at_fault`

In [119]:
df['at_fault'].value_counts()

at_fault
No     29879
Yes    21652
Name: count, dtype: int64

In [120]:
df['at_fault'].isna().sum()

0

No data issues here.

## Column `party_sex`

In [121]:
df['party_sex'].value_counts()

party_sex
Male          31601
Female        14450
Not Stated     4710
Other           770
Name: count, dtype: int64

In [122]:
df['party_sex'].isna().sum()

0

No data issues.

## Column `party_age`

In [123]:
df['party_age'].value_counts()

party_age
 28.0     1219
 30.0     1205
 27.0     1173
 32.0     1172
 25.0     1161
          ... 
-961.0       1
 97.0        1
 108.0       1
-941.0       1
-951.0       1
Name: count, Length: 115, dtype: int64

In [124]:
df['party_age'].isna().sum()

7975

Several data issues here. We observe already some values of `party_age` that are negative. That is not possible, so we will remove those values. Since the column is an integer, we cannot have the value `Not Stated`, so those values will add to the nulls.

In [125]:
df.loc[df['party_age'] < 0, 'party_age'] = None

Also, we should convert the age to integers:

In [126]:
df['party_age'] = df['party_age'].astype(pd.Int64Dtype())

In [127]:
df['party_age'].describe()

count      43519.0
mean     41.623429
std      16.292647
min            0.0
25%           29.0
50%           39.0
75%           53.0
max          109.0
Name: party_age, dtype: Float64

The values shown in describe are always `float`. It is interesting to see that the age values range from 0 to 109. Remember that this are parties involved, so it could be pedestrians, passengers, etc.

## Column `party_sobriety`

In [128]:
df['party_sobriety'].value_counts()

party_sobriety
Had Not Been Drinking                     36010
Not Stated                                 7503
Impairment Not Known                       5201
Had Been Drinking, Not Under Influence     1143
Had Been Drinking, Under Influence          869
Had Been Drinking, Impairment Unknown       698
Not Applicable                               92
Sleepy/Fatigued                               8
Under Drug Influence                          3
Impairment Unknown                            2
Impairment - Physical                         2
Name: count, dtype: int64

In [129]:
df['party_sobriety'].isna().sum()

0

There are a few values to discuss. There are several combinations of drinking and impairements, all having different meanings, so we will keep them. The value `Not Applicable` is not the same as `Not Stated`, it means the column makes no sense (for example, a parked car with no people). Finally, two values have the same meaning with different wording, we will group them:

In [130]:
df.loc[df['party_sobriety'] == 'Impairment Unknown', 'party_sobriety'] = 'Impairment Not Known'

In [131]:
df['party_sobriety'].value_counts()

party_sobriety
Had Not Been Drinking                     36010
Not Stated                                 7503
Impairment Not Known                       5203
Had Been Drinking, Not Under Influence     1143
Had Been Drinking, Under Influence          869
Had Been Drinking, Impairment Unknown       698
Not Applicable                               92
Sleepy/Fatigued                               8
Under Drug Influence                          3
Impairment - Physical                         2
Name: count, dtype: int64

In [132]:
df[df['party_sobriety'] == 'Not Applicable']['party_sex'].value_counts()

party_sex
Not Stated    67
Male          18
Female         7
Name: count, dtype: int64

## Column `party_drug_physical`

In [133]:
df['party_drug_physical'].value_counts()

party_drug_physical
Not Stated                48142
Not Applicable             3028
Under Drug Influence        111
Impairment Not Known         92
Sleepy/Fatigued              87
Impairment - Physical        37
Suspected Cannabis Use       30
Name: count, dtype: int64

In [134]:
df['party_drug_physical'].isna().sum()

4

No data issues here. Given the low quantity of results with values, we will drop this column. 

In [135]:
df.drop(columns='party_drug_physical', inplace=True)

## Column `dir_of_travel`

In [136]:
df['dir_of_travel'].value_counts()

dir_of_travel
East          12507
South         12409
North         12292
West          11532
Not Stated     2791
Name: count, dtype: int64

In [137]:
df['dir_of_travel'].isna().sum()

0

No issues here either.

## Columns  `party_safety_equip_1` and `party_safety_equip_2`

In [138]:
df['party_safety_equip_1'].value_counts().sort_index()

party_safety_equip_1
Air Bag Deployed                  5069
Air Bag Not Deployed             14653
Lap Belt Not Used                   28
Lap Belt Used                      628
Lap/Shoulder Harness Not Used      155
Lap/Shoulder Harness Used         7583
M/C Helmet Driver - No             263
M/C Helmet Driver - Yes            996
M/C Helmet Passenger - No            3
M/C Helmet Passenger - Yes           6
None In Vehicle                    277
Not Required                      6268
Not Stated                       12983
Other                              206
Passive Restraint Not Used           1
Passive Restraint Used               6
Shoulder Harness Not Used           27
Shoulder Harness Used              257
Unknown                           2121
Name: count, dtype: int64

In [139]:
df['party_safety_equip_2'].value_counts().sort_index()

party_safety_equip_2
Air Bag Deployed                         167
Air Bag Not Deployed                     459
Child Restraint in Vehicle Not Used        1
Child Restraint in Vehicle Used            3
Lap Belt Not Used                         72
Lap Belt Used                           1304
Lap/Shoulder Harness Not Used            392
Lap/Shoulder Harness Used              15999
M/C Helmet Driver - No                   767
M/C Helmet Driver - Yes                 2004
M/C Helmet Passenger - No                 16
M/C Helmet Passenger - Yes                22
No Child Restraint in Vehicle              1
None In Vehicle                         1252
None Stated                                1
Not Required                             544
Not Stated                             25481
Other                                      7
Passive Restraint Used                     8
Shoulder Harness Not Used                 44
Shoulder Harness Used                    527
Unknown                           

In [140]:
df['party_safety_equip_1'].isna().sum()

1

We see a differentiation made sometimes between lap and shoulder belts. It is unclear whether Lap/Shoulder means either or both. If it would mean either, we could merge several values together; if it would mean both it would make sense to keep them separated for analysis. We don't know, thus we will keep it separated.

As we have done before, we will replace nulls and `Unknown`/ `None Stated` values with `Not Stated`.

In [141]:
for column in ['party_safety_equip_1', 'party_safety_equip_2']:
    df.loc[df[column].isna(), column] = 'Not Stated'
    df.loc[df[column].isin(['Unknown', 'None Stated']), column] = 'Not Stated'

In [142]:
df['party_safety_equip_1'].value_counts().sort_index()

party_safety_equip_1
Air Bag Deployed                  5069
Air Bag Not Deployed             14653
Lap Belt Not Used                   28
Lap Belt Used                      628
Lap/Shoulder Harness Not Used      155
Lap/Shoulder Harness Used         7583
M/C Helmet Driver - No             263
M/C Helmet Driver - Yes            996
M/C Helmet Passenger - No            3
M/C Helmet Passenger - Yes           6
None In Vehicle                    277
Not Required                      6268
Not Stated                       15105
Other                              206
Passive Restraint Not Used           1
Passive Restraint Used               6
Shoulder Harness Not Used           27
Shoulder Harness Used              257
Name: count, dtype: int64

In [143]:
df['party_safety_equip_2'].value_counts().sort_index()

party_safety_equip_2
Air Bag Deployed                         167
Air Bag Not Deployed                     459
Child Restraint in Vehicle Not Used        1
Child Restraint in Vehicle Used            3
Lap Belt Not Used                         72
Lap Belt Used                           1304
Lap/Shoulder Harness Not Used            392
Lap/Shoulder Harness Used              15999
M/C Helmet Driver - No                   767
M/C Helmet Driver - Yes                 2004
M/C Helmet Passenger - No                 16
M/C Helmet Passenger - Yes                22
No Child Restraint in Vehicle              1
None In Vehicle                         1252
Not Required                             544
Not Stated                             27942
Other                                      7
Passive Restraint Used                     8
Shoulder Harness Not Used                 44
Shoulder Harness Used                    527
Name: count, dtype: int64

## Column `finan_respons`

In [144]:
df['finan_respons'].value_counts()

finan_respons
Yes Proof of Insurance Obtained    29135
Not Applicable                     11389
Not Stated                          9417
No Proof of Insurance Obtained      1463
Name: count, dtype: int64

In [145]:
df['finan_respons'].isna().sum()

127

Only need to fix the nulls:

In [146]:
df.loc[df['finan_respons'].isna(), 'finan_respons'] = 'Not Stated'
df['finan_respons'].isna().sum()

0

## Columns `sp_info_1` , `sp_info_2`, and `sp_info_3`

In [147]:
df['sp_info_1'].value_counts()

sp_info_1
Cell Phone Not In Use                                      27817
Not Stated                                                 22254
Other                                                        591
Cell Phone Handheld In Use                                   278
Cell Phone Handsfree In Use                                  257
Vehicle for Hire (Taxi, Uber, Lyft, etc) WITH Passenger      158
Vehicle for Hire (Taxi, Uber, Lyft, etc) NO Passenger         65
Hazardous Material                                            36
No Cell Phone/Unknown                                         36
32 Ft Trailer Combo                                           18
School Bus Related                                            15
75 Ft Motortruck Combo                                         6
Name: count, dtype: int64

In [148]:
df['sp_info_2'].value_counts()

sp_info_2
Not Stated                                                 39378
Cell Phone Not In Use                                      11504
Vehicle for Hire (Taxi, Uber, Lyft, etc) WITH Passenger      278
Vehicle for Hire (Taxi, Uber, Lyft, etc) NO Passenger        127
Cell Phone Handheld In Use                                    94
Cell Phone Handsfree In Use                                   76
Hazardous Material                                            18
32 Ft Trailer Combo                                           12
School Bus Related                                            11
TNC (Uber, Lyft, etc) WITH Passenger                          11
Other                                                         10
TNC (Uber, Lyft, etc) NO Passenger                             6
75 Ft Motortruck Combo                                         6
Name: count, dtype: int64

In [149]:
df['sp_info_3'].value_counts()

sp_info_3
Not Stated    51529
Other             2
Name: count, dtype: int64

In [150]:
df['sp_info_1'].isna().sum()

0

In [151]:
df['sp_info_2'].isna().sum()

0

In [152]:
df['sp_info_3'].isna().sum()

0

The special info categories do not contain mutually exclusive values, and those found cannot be further grouped. `sp_info_3` has no useful values. `sp_info_1` and `sp_info_2` are a mix of unrelated things like cell usage or whether hazardous materials were involved. The most frequent value in `sp_info_1` is `Cell Phone Not In Use` and the rest are mostly `Not Stated` or some values that could also be cell used or not. Thus, there is limited value in those columns too. We will drop all 3:

In [153]:
drop_columns = ['sp_info_1', 'sp_info_2', 'sp_info_3']
df.drop(columns=drop_columns, inplace=True)

## `oaf` columns

In [154]:
df['oaf_viol_cat'].value_counts()

oaf_viol_cat
Not Stated                     24956
Hit and Run                        3
Failure to Heed Stop Signal        2
Unsafe Speed                       2
Automobile Right-of-Way            1
Other Non-Moving Violation         1
Wrong Side of Road                 1
Name: count, dtype: int64

In [155]:
df['oaf_viol_section'].value_counts()

oaf_viol_section
Not Stated     7280
22350           673
20001(a)        591
12500(a)        173
20001           170
               ... 
22107(a)          1
20001vc           1
21293(a)          1
22100 cvc         1
21453(a) vc       1
Name: count, Length: 513, dtype: int64

In [156]:
df['oaf_violation_code'].value_counts()

oaf_violation_code
Not Stated    45570
Vehicle        4147
20001(a)         10
20002             2
20002(a)          2
22350             2
21460             1
Name: count, dtype: int64

In [157]:
df['oaf_violation_suffix'].value_counts()

oaf_violation_suffix
Not Stated    8206
A                6
Name: count, dtype: int64

In [158]:
df['oaf_1'].value_counts()

oaf_1
None Apparent                  33842
Not Stated                      8235
Violation                       5686
Stop and Go Traffic              925
Inattention                      751
Other                            691
Vision Obscurements              590
Unfamiliar With Road             243
Entering - Leaving Ramp          149
Uninvolved Vehicle               136
Runaway Vehicle                  107
Previous Collision                92
Defective Vehicle Equipment       73
Vision Obscurement                11
Name: count, dtype: int64

In [159]:
df['oaf_2'].value_counts()

oaf_2
Not Stated                     48001
None Apparent                   1318
Other                            124
Stop and Go Traffic              101
Inattention                       75
Runaway Vehicle                   34
Vision Obscurements               28
Unfamiliar With Road              26
Entering - Leaving Ramp           22
Defective Vehicle Equipment       15
Uninvolved Vehicle                13
Previous Collision                11
Violation                         11
Vision Obscurement                 2
Name: count, dtype: int64

There are three columns that have hardly any information, most values are `Not Stated`: `oaf_viol_cat`, `oaf_violation_code`, and `oaf_violation_suffix`. We will start dropping them:

In [160]:
drop_columns = ['oaf_viol_cat', 'oaf_violation_code', 'oaf_violation_suffix']
df.drop(columns=drop_columns, inplace=True)

We will also drop the other columns. While `oaf_viol_section` contains several values, the data does not contain the meaning of the values. The descriptions in `oaf_1` and `oaf_2` should make it easier to read but most are missing. Hence, for the analysis we will drop all remaining "oaf" columns:

In [161]:
drop_columns = ['oaf_viol_section', 'oaf_1', 'oaf_2']
df.drop(columns=drop_columns, inplace=True)

## Column `party_number_killed`

In [162]:
df['party_number_killed'].value_counts()

party_number_killed
0    51313
1      216
2        2
Name: count, dtype: int64

In [163]:
df['party_number_killed'].isna().sum()

0

All good here.

## Column `party_number_injured`

In [164]:
df['party_number_injured'].value_counts()

party_number_injured
1     24711
0     24253
2      2047
3       381
4        92
5        33
6         8
7         4
9         1
13        1
Name: count, dtype: int64

In [165]:
df['party_number_injured'].isna().sum()

0

All good here too.

## Column `move_pre_acc`

In [166]:
df['move_pre_acc'].value_counts()

move_pre_acc
Proceeding Straight                       27744
Making Left Turn                           5403
Stopped                                    4306
Not Stated                                 2743
Parked                                     2605
Making Right Turn                          2194
Other                                      1559
Changing Lanes                              968
Entering Traffic                            834
Slowing/Stopping                            672
Backing                                     628
Making U Turn                               580
Passing Other Vehicle                       250
Traveling Wrong Way                         238
Parking Maneuver                            224
Ran Off Road                                196
Other Unsafe Turning                        124
Merging                                     121
Crossed Into Opposing Lane                  100
Crossed Into Opposing Lane - Unplanned       40
Crossing Into Opposing Lane

In [167]:
df['move_pre_acc'].isna().sum()

0

Those cases with just one occurrence can be changed to same categories with slightly different wording:

In [168]:
df.loc[df['move_pre_acc'] == 'Stopped In Road', 'move_pre_acc'] = 'Stopped'
df.loc[df['move_pre_acc'] == 'Crossing Into Opposing Lane', 'move_pre_acc'] = 'Crossed Into Opposing Lane'

In [169]:
df['move_pre_acc'].value_counts()

move_pre_acc
Proceeding Straight                       27744
Making Left Turn                           5403
Stopped                                    4307
Not Stated                                 2743
Parked                                     2605
Making Right Turn                          2194
Other                                      1559
Changing Lanes                              968
Entering Traffic                            834
Slowing/Stopping                            672
Backing                                     628
Making U Turn                               580
Passing Other Vehicle                       250
Traveling Wrong Way                         238
Parking Maneuver                            224
Ran Off Road                                196
Other Unsafe Turning                        124
Merging                                     121
Crossed Into Opposing Lane                  101
Crossed Into Opposing Lane - Unplanned       40
Name: count, dtype: int64

## Column `vehicle_year`

In [170]:
df['vehicle_year'].value_counts()

vehicle_year
2016.0    2955
2015.0    2887
2017.0    2443
2013.0    2277
2014.0    2151
          ... 
1958.0       1
208.0        1
1916.0       1
66.0         1
1893.0       1
Name: count, Length: 128, dtype: int64

In [171]:
df['vehicle_year'].describe()

count    37487.000000
mean      2000.957265
std        139.714552
min          0.000000
25%       2006.000000
50%       2013.000000
75%       2016.000000
max       5015.000000
Name: vehicle_year, dtype: float64

In [172]:
round(df['vehicle_year'].isna().sum() / df.shape[0], 2)

0.27

Vehicle year clearly has some issues.
- Decimals used when it should be an integer.
- Year as low as 0 (cars were not invented yet!)
- Year as high as 5015 (a Delorian, perhaps? McFly, is that you?)
- 27% of the rows are missing the values (nothing we can do about it here)

Let's tackle first the data type:

In [173]:
df['vehicle_year'] = df['vehicle_year'].astype(pd.Int64Dtype())

How many cars have the wrong values?
- Built in the future
- Built too far in the past (we will be "generous" and assume any car built before year 1900)

In [174]:
f_future = ((df['collision_datetime'].dt.year-df['vehicle_year']) < 0)
f_too_old = (df['vehicle_year'] < 1900)
(f_future | f_too_old).sum()

355

Rules we will use to convert the numbers:
- If the year is between 2023 and 1900 (both included), we do not change the year.
- Otherwise, if the year is between 100 and 199 (1XX), we assume that the intention was to write 19XX.
- Otherwise, if the year's last two numbers are between 0 and 24 (XX), we assume it was year 20XX.
- Otherwise, if the year's last two numbers are between 25 and 99 (XX), we assume it was year 19XX.

- After the previous changes, any year after the `collision_datetime` year, we assume it was a brand new car.


In [175]:
xx = df['vehicle_year']%100
collision_year = df['collision_datetime'].dt.year

# If case
mask_excluded = (df['vehicle_year'] >= 2024) | (df['vehicle_year'] < 1900)

# "otherwise" 1
f1 = mask_excluded & (df['vehicle_year'] >= 100) & (df['vehicle_year'] <= 199)
df.loc[df[f1].index, 'vehicle_year'] = xx + 1900    

# "otherwise" 2
f2 = mask_excluded & (~f1) & (xx < 24)
df.loc[df[f2].index, 'vehicle_year'] = xx + 2000   

# "otherwise" 3
f3 = mask_excluded & (~f1) & (xx >= 24)
df.loc[df[f3].index, 'vehicle_year'] = xx + 1900   

f = collision_year < df['vehicle_year']
df.loc[df[f].index, 'vehicle_year'] = collision_year

In [176]:
f_future = ((df['collision_datetime'].dt.year-df['vehicle_year']) < 0)
f_too_old = (df['vehicle_year'] < 1900)
(f_future | f_too_old).sum()

0

## Column `vehicle_make`

In [177]:
df['vehicle_make'].value_counts()

vehicle_make
TOYOTA                    2627
HONDA                     1623
FORD                      1227
NISSAN                     667
CHEVROLET                  495
                          ... 
WTIN/UNK/WHITE               1
CHEV/CORVETTE/YLW            1
Toyota / Prius /Orange       1
Lexus/GS300/Gold             1
Hyundia/silver               1
Name: count, Length: 20766, dtype: int64

In [178]:
df['vehicle_make'].isna().sum()

7775

In [179]:
df['vehicle_make'].value_counts().iloc[0:40]

vehicle_make
TOYOTA                   2627
HONDA                    1623
FORD                     1227
NISSAN                    667
CHEVROLET                 495
BMW                       448
HYUNDAI                   346
LEXUS                     334
MERCEDES-BENZ             321
VOLKSWAGEN                290
DODGE                     270
UNKNOWN                   261
ACURA                     222
MAZDA                     196
SUBARU                    171
JEEP                      171
AUDI                      151
KIA                       131
GMC                       128
CHRYSLER                  114
BICYCLE                   113
TOYT                      112
YAMAHA                    105
VOLVO                     102
INFINITI                   94
Unknown                    86
MINI                       82
KAWASAKI                   76
Toyota/Prius/White         76
SUZUKI                     71
VESPA                      70
HARLEY-DAVIDSON            68
MITSUBISHI                 

As we can observe, the most frequent values are the overall brand names, and then, a combination of brand, model and even color. We will leave only the brand, as otherwise there are too many different values, and the majority only have the brand. Before anything else, we will convert all text to upper case.

In [180]:
df['vehicle_make'] = df['vehicle_make'].str.upper()

In [181]:
df.loc[df['vehicle_make'] == 'NOT STATED', 'vehicle_make'] = 'UNKNOWN'
df.loc[df['vehicle_make'].isna(), 'vehicle_make'] = 'UNKNOWN'

Let's create a dataframe with those values that have no special characters, and consider those to be the brands.

In [182]:
mask = df['vehicle_make'].str.match(r'^[A-Z]+$') # Strings without special characters
mask = mask & (~(df['vehicle_make'] == 'UNKNOWN')) # do not consider UNKNOWN as a brand
vehicle_brands = df[mask]['vehicle_make'].value_counts().index # Unique values
vehicle_brands.shape[0]

573

Next, we try to find for each value that is not identified as a brand, if it contains one of the brands (prioritizing those with higher frequency). This has some issues. For example, `GRAY` has been identified as one of the most common brands (many officers simply write the color) and thus, any brand that has appeared less frequently than the word gray, will be misidentified as brand `GRAY`. 

In [183]:
import re
for brand in vehicle_brands:
    pattern = r'(?<![A-Z])' + re.escape(brand) + r'(?![A-Z])'
    mask_unbranded = ~df['vehicle_make'].isin(vehicle_brands)
    mask_replace = mask_unbranded & df['vehicle_make'].str.contains(pattern, regex=True) 
    df.loc[mask_replace, 'vehicle_make'] = brand

In [184]:
df['vehicle_make'].value_counts()

vehicle_make
UNKNOWN               8133
TOYOTA                7246
HONDA                 5021
FORD                  3542
NISSAN                1931
                      ... 
MUNIBUS                  1
WIDEWHEEL  MERCANE       1
MERZU                    1
HINDA                    1
ALLWAY YA309C/D          1
Name: count, Length: 943, dtype: int64

In [185]:
df['vehicle_make'].value_counts().shape[0]

943

Next we replace values with similar ones, progressively to approach the most frequent appearing:

In [186]:
# replace similar values:
for threshold in [0.9, 0.8, 0.7]:
    values = df['vehicle_make'].value_counts().index
    for i in reversed(range(len(values))):
        old_value = values[i]
        for j in range(0, i):
            new_value = values[j]
            ratio = difflib.SequenceMatcher(None, old_value, new_value).ratio()
            if ratio > threshold and ratio < 1:
                print("[" + str(threshold) + "] " + old_value + " new: " + new_value)            
                df.loc[df['vehicle_make'] == old_value, 'vehicle_make'] = new_value
                break

[0.9] GILLI new: GILLIG
[0.9] GENZIE new: GENZE
[0.9] SPECIALLIZED new: SPECIALIZED
[0.9] FORK LIFT new: FORKLIFT
[0.9] NINE BOT new: NINEBOT
[0.9] PETERBUILT new: PETERBILT
[0.9] NISSA new: NISSAN
[0.9] KALKHOFF new: KAL KHOFF
[0.9] XTRACYLE new: XTRACYCLE
[0.9] BICYLCE new: BICYLE
[0.9] PINARELO new: PINARELLO
[0.9] CANNON DALE new: CANNONDALE
[0.9] HON/CIV/SIL new: HON/CRV/SIL
[0.9] CHEVEROLET new: CHEVROLET
[0.9] GENEZE new: GENZE
[0.9] PIAGGO new: PIAGGIO
[0.9] TOYOTOA new: TOYOTA
[0.9] RALEIGHT new: RALEIGH
[0.9] DIAMONDB new: DIAMOND
[0.9] CENTURIO new: CENTURION
[0.9] CANONDAL new: CANONDALE
[0.9] CHRYLER new: CHRYSLER
[0.9] HUNDAI new: HYUNDAI
[0.9] CHYSLER new: CHRYSLER
[0.9] PUEGOT new: PUEGEOT
[0.9] BICYLE new: BICYCLE
[0.9] PEUGOT new: PEUGEOT
[0.9] SPECLZED new: SPECLIZED
[0.9] SURLEY new: SURLY
[0.9] CADILAC new: CADILLAC
[0.9] SKATEBOARDER new: SKATEBOARD
[0.9] (REFER TO NARRATIVE) new: REFER TO NARRATIVE
[0.9] MYATA new: MIYATA
[0.9] ANCHER new: ANCHEER
[0.9] LAND ROVE

In [187]:
df['vehicle_make'].value_counts().head(20)

vehicle_make
TOYOTA       8426
UNKNOWN      8138
HONDA        5267
FORD         3544
NISSAN       2154
BLACK        1448
BMW          1394
LEXUS        1111
HYUNDAI      1023
CHEVY         988
WHITE         920
DODGE         851
CHEVROLET     765
BICYCLE       644
MAZDA         627
ACURA         616
SUBARU        606
SCOOTER       597
MERCEDES      561
JEEP          536
Name: count, dtype: int64

In [188]:
df['vehicle_make'].value_counts().shape[0]

663

It is likely this caused some miscategorizations, and the result could be further improved by manually checking values. Nevertheless, it is in better shape than the original data. We stop it here.

Whereas `UNKNOWN` was easier to work with, we will change that to the usual `Not Stated`: 

In [189]:
df.loc[df['vehicle_make'] == 'UNKNOWN', 'vehicle_make'] = 'Not Stated'

## Column `stwd_vehicle_type`

In [190]:
df['stwd_vehicle_type'].value_counts()

stwd_vehicle_type
Passenger Car                                                                       30266
Pedestrian                                                                           6252
Bicycle                                                                              4020
Sport Utility Vehicle                                                                2480
Motorcycle                                                                           2171
                                                                                    ...  
Dune Buggy                                                                              1
Pickup and Camper (Hazardous Waste or Hazardous Waste/Material Combination)             1
Trailer Coach                                                                           1
Container Chassis                                                                       1
Truck or Truck Tractor (Hazardous Waste or Hazardous Waste/Material Combination)  

In [191]:
df['stwd_vehicle_type'].isna().sum()

0

In [192]:
df['stwd_vehicle_type'].value_counts().head(40)

stwd_vehicle_type
Passenger Car                               30266
Pedestrian                                   6252
Bicycle                                      4020
Sport Utility Vehicle                        2480
Motorcycle                                   2171
Pickup or Panel Truck                        1317
Motor Driven Cycle/Scooter                    902
Go-ped, ZIP Electric Scooter, Motorboard      807
Minivan                                       761
Other Bus                                     332
Unknown Hit and Run                           180
Public Transit Authority                      162
Paratransit                                   157
All Terrain Vehicle                           149
Two Axle Truck                                140
Other Commercial                              134
Not Stated                                    131
Truck                                         128
Emergency Vehicle                             126
Police Car                      

`Unknown` and `Not Stated` are values. We will merge them:

In [193]:
df.loc[df['stwd_vehicle_type'] == 'Unknown', 'stwd_vehicle_type'] = 'Not Stated'

## Column `race`

In [194]:
df['race'].value_counts()

race
White         15754
Hispanic       9549
Asian          8737
Black          6763
Not Stated     6083
Other          4645
Name: count, dtype: int64

In [195]:
df['race'].isna().sum()

0

Nothing to do here.

## Column `inattention`

In [196]:
df['inattention'].value_counts()

inattention
Not Stated                     49947
Cell Phone Not In Use           1392
Other                             89
Cellphone Handheld                24
Cell Phone Handsfree In Use       15
Cell Phone Handheld In Use        10
Electronic Equipment               7
Eating                             5
Cellphone Handsfree                5
Hazardous Material                 5
32 Ft Trailer Combo                5
Animals                            4
Smoking                            2
Radio/CD                           2
School Bus Related                 1
Children                           1
Unknown                            1
Name: count, dtype: int64

Another column with very few values, we will drop it.

In [197]:
df.drop(columns='inattention', inplace=True)

## Columns `special_info_f` and `special_info_g`

In [198]:
df['special_info_f'].value_counts()

special_info_f
Not Stated    16804
Name: count, dtype: int64

In [199]:
df['special_info_g'].value_counts()

special_info_g
Not Stated    16804
Name: count, dtype: int64

No information here, we drop the columns.

In [200]:
df.drop(columns=['special_info_f', 'special_info_g'], inplace=True)

## Column `street_of_travel`

In [201]:
df['street_of_travel'].value_counts()

street_of_travel
MISSION ST            1338
MARKET ST              854
GEARY BLVD             626
MISSION STREET         609
VAN NESS AVE           576
                      ... 
44TH AVE FULTON          1
MISSION STREET`          1
28TH                     1
HUNTERS EXPRESSWAY       1
CLARENDON AVENUE         1
Name: count, Length: 3861, dtype: int64

In [202]:
df['street_of_travel'].isna().sum()

2022

Similar adjustments could be done like for columns `primary_rd` and `secondary_rd`. We only adjust the nulls:

In [203]:
df.loc[df['street_of_travel'].str.upper() == 'UNKNOWN', 'street_of_travel'] = 'Not Stated'
df.loc[df['street_of_travel'].str.upper() == 'NOT STATED', 'street_of_travel'] = 'Not Stated'
df.loc[df['street_of_travel'].isna(), 'street_of_travel'] = 'Not Stated'

## Column `vehicle_autonomous`

In [204]:
df['vehicle_autonomous'].value_counts()

vehicle_autonomous
Not applicable, conventional vehicle                                       18856
Not Stated                                                                 16378
Not Collected                                                              16192
Autonomous vehicle operating under driver control at time of collision        76
Operating in autonomous mode at time of collision with a driver present       11
Driverless vehicle with passenger only, no driver override possible           11
Autopilot engaged (autoparking, etc.) with driver present                      7
Name: count, dtype: int64

There are hardly any autonomous vehicle to be a meaningful sample, so we will remove the column.

In [205]:
df.drop(columns='vehicle_autonomous', inplace=True)

## Column `Current Police Districts`

In [206]:
df['Current Police Districts'].value_counts().sort_index()

Current Police Districts
1.0     7126
2.0     5214
3.0     7315
4.0     7862
5.0     2869
6.0     4671
7.0     2247
8.0     3809
9.0     5014
10.0    5238
Name: count, dtype: int64

In [207]:
df['Current Police Districts'].isna().sum()

166

The values are as expected, but should be converted to Integers:

In [208]:
df['Current Police Districts'] = df['Current Police Districts'].astype(pd.Int64Dtype())

## Column `Current Supervisor Districts`

In [209]:
df['Current Supervisor Districts'].value_counts().sort_index()

Current Supervisor Districts
1.0      3254
2.0      5487
3.0      4487
4.0      3524
5.0      3857
6.0      3508
7.0      1730
8.0      3017
9.0      5799
10.0    10911
11.0     5807
Name: count, dtype: int64

In [210]:
df['Current Supervisor Districts'].isna().sum()

150

Like the previous column, we need to convert to integer:

In [211]:
df['Current Supervisor Districts'] = df['Current Supervisor Districts'].astype(pd.Int64Dtype())

# New column: `vehicle_age`
To facilitate the analysis, we create a new columm `vehicle_age` as the difference between the year of the collision, and the `vehicle_year`:

In [212]:
df['vehicle_age'] = (df['collision_datetime'].dt.year-df['vehicle_year']).astype(pd.Int64Dtype())

# Format output

To prepare the DataFrame to export as a CSV, we will reorder the columns and export as CSV.

In [213]:
columns_reordered = ['case_id_pkey', 'tb_latitude', 'tb_longitude', 'collision_datetime',
       'officer_id', 'primary_rd', 'secondary_rd', 'distance', 'direction',
       'weather_1', 'weather_2', 'collision_severity', 'type_of_collision',
       'mviw', 'ped_action', 'road_surface', 'road_cond_1', 'road_cond_2',
       'lighting', 'control_device', 'vz_pcf_description',
       'dph_col_grp_description', 'party_number_ckey', 'party_type',
       'at_fault', 'party_sex', 'party_age', 'race', 'party_sobriety',
       'party_safety_equip_1', 'party_safety_equip_2', 'finan_respons',
       'party_number_killed', 'party_number_injured', 'move_pre_acc',
       'vehicle_year', 'vehicle_make', 'vehicle_age', 'stwd_vehicle_type',
       'dir_of_travel', 'street_of_travel', 'Current Police Districts',
       'Current Supervisor Districts']

In [214]:
file_name_output = 'sf_traffic.csv'

In [215]:
df[columns_reordered].to_csv(file_name_output, sep=',')